In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('simulated_radar_data.csv')
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   object_type  3000 non-null   object 
 1   range        3000 non-null   float64
 2   velocity     3000 non-null   float64
 3   angle        3000 non-null   float64
 4   amplitude    3000 non-null   float64
dtypes: float64(4), object(1)
memory usage: 117.3+ KB


,range,velocity,angle,amplitude
count,3000.000000,3000.000000,3000.000000,3000.00000
mean,25.012483,8.087037,11.519387,0.11661
std,4.115688,5.667409,14.318247,0.11907
min,19.110000,0.550000,-8.310000,-0.30000
25%,20.200000,1.330000,-4.360000,0.04000
50%,25.000000,8.020000,10.010000,0.12000
75%,29.812500,14.880000,29.290000,0.20000
max,30.970000,15.580000,32.930000,0.54000


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Features and target
X = df[['range', 'velocity', 'angle', 'amplitude']]
y = df['object_type']

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42
)

# Train MLP model
mlp = MLPClassifier(hidden_layer_sizes=(2,), activation="relu", solver="adam", max_iter=54, random_state=42)
mlp.fit(X_train, y_train)

# Predict and evaluate
y_pred = mlp.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))


              precision    recall  f1-score   support

        bird       1.00      0.99      1.00       207
         car       0.99      1.00      0.99       189
       human       1.00      1.00      1.00       204

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



c:\Work\Python\NeuralNetwrkOC\radarml\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (54) reached and the optimization hasn't converged yet.
  warnings.warn(


In [8]:
# Weights from input to hidden layer
input_to_hidden_weights = mlp.coefs_[0]      # shape: (4 features, 2 hidden neurons)
hidden_biases = mlp.intercepts_[0]           # shape: (2,)

# Weights from hidden to output layer
hidden_to_output_weights = mlp.coefs_[1]     # shape: (2 hidden neurons, 3 classes)
output_biases = mlp.intercepts_[1]           # shape: (3,)

print("Input to Hidden Weights:\n", input_to_hidden_weights)
print("Hidden Biases:\n", hidden_biases)

print("Hidden to Output Weights:\n", hidden_to_output_weights)
print("Output Biases:\n", output_biases)

Input to Hidden Weights:
 [[-0.65015046  1.59786261]
 [ 0.0704379   0.89574153]
 [-1.15100561 -1.03905404]
 [-0.50363138  0.71642018]]
Hidden Biases:
 [0.36736791 0.95864263]
Hidden to Output Weights:
 [[-1.20218887  0.44200463  1.36797478]
 [-1.1090307  -0.24417826 -1.12439868]]
Output Biases:
 [ 0.22493537 -0.54368607 -0.50191126]


In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 1. Define the same model architecture
model = Sequential([
    Dense(2, input_shape=(4,), activation='relu', name='hidden'),
    Dense(3, activation='softmax', name='output')  # or another activation if originally different
])

# 2. Manually set weights
input_to_hidden_weights = np.array([[-0.65, 1.59],
                                    [ 0.07 , 0.89],
                                    [-1.15, -1.03],
                                    [-0.50, 0.71]])

hidden_biases = np.array([0.36, 0.95])

hidden_to_output_weights = np.array([[-1.20,  0.44,  1.36],
                                     [-1.10 , -0.24, -1.12]])

output_biases = np.array([ 0.22, -0.54, -0.50])

# Assign weights to layers
model.get_layer('hidden').set_weights([input_to_hidden_weights, hidden_biases])
model.get_layer('output').set_weights([hidden_to_output_weights, output_biases])

# 3. Inference
sample_input = np.array([[20.38,1.21,-4.02,0.06]])  # replace with your input
output = model.predict(sample_input)
print(output)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[[8.0888030e-15 1.0000000e+00 1.8188313e-15]]


In [12]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

def evaluate_custom_mlp(X, y, weights_dict, is_categorical=False):
    """
    Evaluate accuracy of a manually-weighted MLP model.

    Parameters:
        X (np.ndarray): Input data, shape (num_samples, 4)
        y (np.ndarray): Labels, shape (num_samples,) or one-hot (num_samples, 3)
        weights_dict (dict): Dictionary with keys:
            'input_hidden_weights', 'hidden_biases',
            'hidden_output_weights', 'output_biases'
        is_categorical (bool): Set True if y is one-hot encoded

    Returns:
        float: Accuracy score
    """

    # 1. Build model with matching architecture
    model = Sequential([
        Dense(2, input_shape=(4,), activation='relu', name='hidden'),
        Dense(3, activation='softmax', name='output')
    ])

    # 2. Set custom weights
    model.get_layer('hidden').set_weights([
        np.array(weights_dict['input_hidden_weights']),
        np.array(weights_dict['hidden_biases'])
    ])

    model.get_layer('output').set_weights([
        np.array(weights_dict['hidden_output_weights']),
        np.array(weights_dict['output_biases'])
    ])

    # 3. Get predictions
    predictions = model.predict(X)
    predicted_classes = np.argmax(predictions, axis=1)

    # 4. Process ground truth
    if is_categorical:
        true_classes = np.argmax(y, axis=1)
    else:
        true_classes = y

    # 5. Compute accuracy
    accuracy = accuracy_score(true_classes, predicted_classes)
    return accuracy


In [21]:
weights = {
    "input_hidden_weights" : [[-0.65,  1.59],
 [ 0.07,   0.89],
 [-1.15, -1.03],
 [-0.50,  0.71],],
    "hidden_biases" : [0.36, 0.95],
    "hidden_output_weights" : [[-1.20,  0.44,  1.36],
 [-1.10,  -0.24, -1.12]],
    "output_biases" : [ 0.22, -0.54, -0.50]
}

accuracy = evaluate_custom_mlp(X_scaled, y_encoded, weights, is_categorical=False)
print("Accuracy:", accuracy)

24/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

c:\Work\Python\NeuralNetwrkOC\radarml\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Accuracy: 0.9963333333333333
